# Object Detection(객체 탐지)
<hr>

- 두 가지 작업(task)을 포함한 것을 `Object Detection`이라고 함
    - Localization(위치결정): 물체(객체)의 `위치 정보`를 찾는 것
        - `위치 정보`: 더 정확하게는 물체(객체)가 있을 것 같은 `경계상자`를 찾는 것
    - Classification(분류): 해당 상자에 포함되어 있는 물체(객체)가 `무엇인지 분류`하는 것
    - Localization과 Classification을 수행하기 위해서 일반적으로 두 단계가 필요
        - 1단계. 물체(객체)가 있을 만한 `경계상자`를 추출
            - `경계상자`: resions of interest, `ROI`(관심 영역)
        - 2단계. 추출된 `ROI`를 이용하여 Localization과 Classification을 수행
    - Object Detection 알고리즘은 크게 두 가지로 구분됨
        - `One Stage Detectors(or Algorithm)`
            - 위 두 단계를 한 번에 수행
            - 알고리즘: SSD(Single Shot Detection), YOLO(You Only Look Once) 등
            - 특징: 상대적으로 연산 속도가 빠르고, 정확도가 낮지만 큰 차이는 없거나 더 좋은 경우도 있음
            - 참고사항: 대표적인 두 알고리즘 중 SSD가 일반적으로 성능이 더 좋다고 알려져 있어,<br>
            더 많이 사용됨. SSD가 작은 사이즈의 객체를 탐지하는 성능이 YOLO보다 우수하다고 알려져 있음
        - `Two Stage Detectors(or Algorithm)`
            - 위 두 단계를 구분하여 수행
            - 알고리즘: `R-CNN` family(R-CNN, Fast R-CNN, Faster R-CNN, `Mask R-CNN`)
                - `R-CNN`: Region-Based Convolutional Neural Network(영역 기반 합성곱 신경망)
                - `Mask R-CNN`: 물체(객체) 탐지도 수행하지만 주 목적은 `Image Segmentation`(이미지 분할)
            - 특징: 상대적으로 연산 속도가 느리고, 정확도가 높다.
- Object Detection은 지도학습과 비지도학습 중 `지도학습`에 해당함
    - Localization과 Classification 이 두가지 작업에 해당하기 때문에 각각에 대한 레이블이 필요함

### SSD(Single Shot Detection)
<hr>

- 지도학습의 한 종류이기 때문에 이미지 안에 정답 경계상자, 클래스 정보가 담긴 `정답 이미지`가 존재해야 하며,<br>
알고리즘이 `정답 경계상자`를 찾고, 각 `정답 경계상자`에 속한 객체를 예측해야 함
    - `정답 경계상자`: Ground Truth Bounding box, `GTBB`
- 이미지마다 존재하는 객체의 수, 객체의 위치, 객체의 크기, 객체의 형태가 다르다. 그러므로<br>
아무것도 없는 상태의 이미지에서 위치, 크기, 형태가 각각 다른 GTBB를 예측하는 것은 어렵기 때문에<br>
`기본상자`라는 것을 사용
    - `기본상자`: Anchor box or Default box
    - `참고사항`: Faster R-CNN에서 이 기본상자를 활용한 `RPN(Resion Proposal Network)`방식을 처음 도입,<br>
    이후 SSD에서 RPN 단계를 Classification과 통합하여 수행함
- Anchor box(AB)
    - 다양한 위치, 크기, 형태의 AB를 사용하여 AB를 기준점으로 GTBB가 AB로부터 얼마나 떨어져 있는지(`offset` 정도),<br>
    그리고 해당 GTBB에 존재하는 객체가 무엇인지를 예측함(Localization, Classification의 두 작업 동시 수행)
    - SSD는 AB를 원본 이미지에서 생성하지 않고 `사전학습모형(VGG-16)`을 이용하여 특징 맵(Feature map)을 추출하고,<br>
    추출된 특징 맵(Feature map)에서 AB를 생성하여 사용함(이후 offset과 class를 예측)
        - `사전 학습 모형`: backbone network라고 일컬음
- Feature map(특징 맵)
    - SSD는 Object Detection을 위해 사용하는 `특징 맵의 수가 6개`
    - 1. 1차적으로 VGG-16과 같은 사전학습모형(backbone network)을 사용하여 6개의 특징 맵을 추출하고,<br>
    그 중 1개를 사용
    - 2. 추가적인 합성곱 레이어를 사용하여 6개의 특징 맵을 추출하고, 그 중 5개를 사용
    - 결론적으로 `사전학습모형(1개) + 합성곱 레이어(5개) = 6개의 크기가 각각 다른 특징 맵`을 사용하고,<br>
    이를 이용하여 물체(객체)를 탐지하여 다양한 크기의 물체(객체)를 잘 찾을 수 있음
    - <img src="markdown/SSD_architecture.png", width="600">